In [57]:
import numpy as np
import random


In [2]:
class VernamError(Exception):
    pass

In [49]:
class Vernam:
    def __init__(self, key: str):
        try:
            self.key = bytes.fromhex(key)
        except ValueError:
            raise VernamError("Wrong format key entered (Hex)")
    
    @staticmethod
    def gen_key(size: int) -> str:
        sample = tuple(np.random.randint(0, 256, size))
        return bytes(sample).hex()
    
    def _transform(self, data: bytes or str, mode: str = "encrypt"):
        if mode not in ("encrypt", "decrypt"):
            raise VernamError("!")
        
        if not data:
            raise VernamError("Input text is empty!")

        if not self.key:
            raise VernamError("The key is missing!")
        
        # check input data
        match mode, data:
            case "encrypt", str():
                data_bytes = bytearray(data, "utf-8")
                
            case "decrypt", str():
                data_bytes = bytearray.fromhex(data)
            
            case _, bytes():
                data_bytes = bytearray(data)
                
            case _:
                raise VernamError(f"Invalid processing type! -> {mode}")

        if len(self.key) != len(data_bytes):
            raise VernamError(f"Key size ({len(self.key)}) and text size ({len(data_bytes)}) in bytes must match!")

        for i in range(len(data_bytes)):
            data_bytes[i] ^= self.key[i % len(self.key)]
                
        # manage output
        match mode, data:
            case "encrypt", str():
                return data_bytes.hex()
                
            case "decrypt", str():
                return data_bytes.decode("utf-8")
            
            case _, bytes():
                return bytes(data_bytes)
                
            case _:
                raise VernamError(f"Invalid processing type! -> {mode}")
        
    def encrypt(self, data: str or bytes):
        return self._transform(data, "encrypt")
    
    def decrypt(self, data: str or bytes):
        return self._transform(data, "decrypt")

In [50]:
message = "Hello, World! Привет, Мир! 🐼 🛠 難"

In [51]:
key = Vernam.gen_key(len(bytes(message, "utf-8")))
vernam = Vernam(key)

In [52]:
encrypted_message = vernam.encrypt(message)
print("encrypted message: ", encrypted_message)

encrypted message:  c35667ee2775b6163f2d8a2c9afd5e2f2a7a98eb88e93cf2092f55e9afaae1610d09f887392b3ac5fac51ed1f0739c4659


In [53]:
decrypted_message = vernam.decrypt(encrypted_message)
print("decrypted message: ", decrypted_message)

decrypted message:  Hello, World! Привет, Мир! 🐼 🛠 難


In [54]:
message_2 = bytes(message, "utf-8")
encrypted_message_2 = vernam.encrypt(message_2)
print("encrypted message: ", encrypted_message_2)

encrypted message:  b"\xc3Vg\xee'u\xb6\x16?-\x8a,\x9a\xfd^/*z\x98\xeb\x88\xe9<\xf2\t/U\xe9\xaf\xaa\xe1a\r\t\xf8\x879+:\xc5\xfa\xc5\x1e\xd1\xf0s\x9cFY"


In [55]:
decrypted_message_2 = vernam.decrypt(encrypted_message_2)
print("decrypted message: ", decrypted_message_2)

decrypted message:  b'Hello, World! \xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82, \xd0\x9c\xd0\xb8\xd1\x80! \xf0\x9f\x90\xbc \xf0\x9f\x9b\xa0 \xe9\x9b\xa3'


In [56]:
decrypted_message_2.decode("utf-8")

'Hello, World! Привет, Мир! 🐼 🛠 難'